In [2]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
%pip install numpy torch nltk matplotlib Flask flask-socketio


     ------------------------------------ 198.5/198.5 MB 793.5 kB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
Note: you may need to restart the kernel to use updated packages.


In [13]:
lemmatizer = WordNetLemmatizer()

In [14]:
intents = json.loads(open('intents.json').read())

In [15]:
from tensorflow.keras import models

In [16]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = models.load_model('chat_model.h5')

In [17]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words  = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [18]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]* len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)
    

In [19]:
def predict_classes(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result =[[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    result.sort(key = lambda x: x[1], reverse =  True)
    return_list = []
    for r in result:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [20]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents =  intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [4]:
print("Go! bot is Running!")
while True:
    message = input("You: ")
    ints = predict_classes(message)
    res = get_response(ints,intents)
    print(res)

Go! bot is Running!


NameError: name 'predict_classes' is not defined